## Deep Learning Model - CNN Multi-Label Text Classification
### GLOVE with Sub Themes (Right Now Ran for Main Themes)

In [1]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv1D, Conv2D, MaxPooling2D, GlobalMaxPooling1D, MaxPool1D, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM
from keras.utils import to_categorical
from keras import backend as K


import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import spacy
# from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

Using plaidml.keras.backend backend.


### Loading Data

In [2]:
X_train_Q1 = pd.read_excel('../data/interim/X_train_Q1_clean.xlsx')
X_valid_Q1 = pd.read_excel('../data/interim/X_valid_Q1_clean.xlsx')

y_train_Q1 = pd.read_excel('../data/interim/y_train_Q1.xlsx')
y_valid_Q1 = pd.read_excel('../data/interim/y_valid_Q1.xlsx')

### Undersampling Data

### Creating a Unified Dataframe for CNN Ready Model

In [3]:
df = pd.concat([X_train_Q1, y_train_Q1.iloc[:,:12]], axis = 1)

In [4]:
df.head()

,Comment,CPD,CB,EWC,Exec,FEW,SP,RE,Sup,SW,TEPE,VMG,OTH
0,"to be real about diversity, you need to create...",0,0,1,0,0,0,0,0,0,0,0,0
1,Keep the building warmer and provide warm wate...,0,0,0,0,0,0,0,0,0,1,0,0
2,better communication from the top down,0,0,0,1,0,0,0,0,0,0,0,0
3,It would be beneficial if Management did not m...,0,0,0,0,0,0,1,0,0,0,0,0
4,more education applicable to my job,1,0,0,0,0,0,0,0,0,0,0,0


In [5]:
data_df = df

In [6]:
data_df.shape

(10376, 13)

### Pre-processing

#### General

In [7]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [8]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves',\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [9]:
from tqdm import tqdm
preprocessed_synopsis = []
# tqdm is for printing the status bar
for sentance in data_df['Comment'].values:
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_synopsis.append(sentance.strip())
data_df['preprocessed_comments']=preprocessed_synopsis

In [10]:
data_df.head()

,Comment,CPD,CB,EWC,Exec,FEW,SP,RE,Sup,SW,TEPE,VMG,OTH,preprocessed_comments
0,"to be real about diversity, you need to create...",0,0,1,0,0,0,0,0,0,0,0,0,real diversity you need create seats table mea...
1,Keep the building warmer and provide warm wate...,0,0,0,0,0,0,0,0,0,1,0,0,keep building warmer provide warm water bathroom
2,better communication from the top down,0,0,0,1,0,0,0,0,0,0,0,0,better communication top
3,It would be beneficial if Management did not m...,0,0,0,0,0,0,1,0,0,0,0,0,would beneficial management not micro manage
4,more education applicable to my job,1,0,0,0,0,0,0,0,0,0,0,0,education applicable job


### Splitting into Train and Test

In [11]:
X_train = data_df[['preprocessed_comments']]
y_train = data_df.drop(['Comment', 'preprocessed_comments'], axis=1)

### Preparing Labels

**Max length of sentence**

In [12]:
def max_len(x):
    a=x.split()
    return len(a)

max_len = max(data_df['Comment'].apply(max_len))
max_len

150

### Vocab Size

**For General Use**

In [13]:
vect=Tokenizer()
vect.fit_on_texts(X_train['preprocessed_comments'])
vocab_size = len(vect.word_index) + 1
print(vocab_size)

11919


**Glove**

In [14]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('/Users/karan/Downloads/glove/glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [15]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in vect.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [16]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.11619   ,  0.45447001, -0.69216001, ..., -0.54737002,
         0.48822001,  0.32246   ],
       [-0.19103999,  0.17601   ,  0.36919999, ..., -0.59680003,
         0.080843  ,  0.27866   ],
       ...,
       [-0.34926   ,  0.27006999, -0.52661002, ...,  0.22747   ,
        -0.12559   ,  0.70643002],
       [-0.53812999,  0.72706997,  0.074018  , ..., -0.41005999,
         1.08850002,  0.75314999],
       [-1.51540005,  0.66566002,  0.23134001, ...,  0.47402   ,
         0.84129   ,  0.94787002]])

In [23]:
embedding_matrix.shape

(11919, 100)

In [17]:
# saving this embedding

np.save('../models/glove_embedding_100d', embedding_matrix)

## Modelling CNN

#### Padding to make all sequences of same length

**Training Data**

In [18]:
encoded_docs_train = vect.texts_to_sequences(X_train['preprocessed_comments'])
max_length = vocab_size
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=max_len, padding='post')
print(padded_docs_train)

[[  504   585    36 ...     0     0     0]
 [  134    54  3393 ...     0     0     0]
 [    7    29   234 ...     0     0     0]
 ...
 [  697     4    12 ...     0     0     0]
 [  476  2745 11917 ...     0     0     0]
 [ 1147   593   791 ...     0     0     0]]


In [19]:
padded_docs_train.shape

(10376, 150)

In [20]:
# saving padded training data

np.save('../models/glove_x_train_padded', padded_docs_train)

#### Defining Model

## Glove

In [24]:
y_train = np.array(y_train)

In [25]:
max_features = embedding_matrix.shape[0]
maxlen = max_len
batch_size = 128
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 10
embed_size = 100 # for glove we are using 100d dataset
n_class = 12

In [26]:
model = Sequential()

model.add(Embedding(max_features, embed_size, weights=[embedding_matrix],
                        trainable=False, input_length=maxlen))

model.add(Dropout(0.2))
model.add(Conv1D(filters, kernel_size, padding='valid', activation='relu',
                 strides=1))
model.add(MaxPooling1D())
model.add(Conv1D(filters, kernel_size, padding='valid',activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(hidden_dims, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(n_class, activation = 'sigmoid'))


model.summary()

INFO:plaidml:Opening device "metal_amd_radeon_pro_555x.0"


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 100)          1191900   
_________________________________________________________________
dropout_1 (Dropout)          (None, 150, 100)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 148, 250)          75250     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 74, 250)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 72, 250)           187750    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 36, 250)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9000)              0         
__________

In [24]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
# Train Model
model.fit(padded_docs_train, y_train, batch_size=batch_size, epochs=epochs,
          validation_split=0.15)

Train on 8819 samples, validate on 1557 samples
Epoch 1/10
8819/8819 [==============================] - 978s 111ms/step - loss: 0.3699 - acc: 0.8777 - val_loss: 0.3105 - val_acc: 0.8949
Epoch 2/10
1408/8819 [===>..........................] - ETA: 10:53 - loss: 0.3185 - acc: 0.8949

/Users/karan/anaconda3/lib/python3.7/site-packages/numpy/ctypeslib.py:436: RuntimeWarning: Invalid PEP 3118 format string: '&<f'
  return array(obj, copy=False)


8819/8819 [==============================] - 1004s 114ms/step - loss: 0.3042 - acc: 0.8955 - val_loss: 0.2673 - val_acc: 0.9028
Epoch 3/10
8819/8819 [==============================] - 950s 108ms/step - loss: 0.2660 - acc: 0.9032 - val_loss: 0.2369 - val_acc: 0.9128
Epoch 4/10
8819/8819 [==============================] - 1050s 119ms/step - loss: 0.2347 - acc: 0.9148 - val_loss: 0.2218 - val_acc: 0.9182
Epoch 5/10
8819/8819 [==============================] - 951s 108ms/step - loss: 0.2152 - acc: 0.9212 - val_loss: 0.2140 - val_acc: 0.9213
Epoch 6/10
8819/8819 [==============================] - 1005s 114ms/step - loss: 0.2004 - acc: 0.9264 - val_loss: 0.2045 - val_acc: 0.9259
Epoch 7/10
8819/8819 [==============================] - 1018s 115ms/step - loss: 0.1877 - acc: 0.9308 - val_loss: 0.2002 - val_acc: 0.9272
Epoch 8/10
8819/8819 [==============================] - 1019s 116ms/step - loss: 0.1752 - acc: 0.9347 - val_loss: 0.1982 - val_acc: 0.9277
Epoch 9/10
8819/8819 [==================

**Evaluation on Validation data**

In [21]:
df_valid = pd.concat([X_valid_Q1, y_valid_Q1.iloc[:,:12]], axis = 1)

# pre-processing test data
from tqdm import tqdm
preprocessed_synopsis = []
# tqdm is for printing the status bar
for sentance in df_valid['Comment'].values:
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_synopsis.append(sentance.strip())
    
    
df_valid['preprocessed_comments']=preprocessed_synopsis

# creating X and Y
X_valid = df_valid[['preprocessed_comments']]
y_valid = df_valid.drop(columns=['Comment', 'preprocessed_comments'])

y_valid = np.array(y_valid)
# creating padded dataset for x_valid
encoded_docs_valid = vect.texts_to_sequences(X_valid['preprocessed_comments'])
padded_docs_valid = pad_sequences(encoded_docs_valid, maxlen=max_len, padding='post')

In [22]:
# saving padded test data
np.save('../models/glove_x_valid_padded', padded_docs_valid)

In [ ]:

score = model.evaluate(padded_docs_valid,y_valid)
score

## Saving Model

In [27]:
model.save('../models/cnn_glove_themes.pkl')

## Precision & Recall

In [28]:
pred = model.predict(padded_docs_valid, batch_size=batch_size, verbose=1)

2594/2594 [==============================] - 226s 87ms/step


In [29]:
from sklearn.metrics import precision_score, recall_score, f1_score

#predictions=model.predict([padded_docs_test])
predictions = pred
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_valid, pred, average='micro')
    recall = recall_score(y_valid, pred, average='micro')
    f1 = f1_score(y_valid, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))
    print('\n')

For threshold:  0.1
Micro-average quality numbers
Precision: 0.4251, Recall: 0.8477, F1-measure: 0.5663


For threshold:  0.2
Micro-average quality numbers
Precision: 0.5672, Recall: 0.7510, F1-measure: 0.6463


For threshold:  0.3
Micro-average quality numbers
Precision: 0.6644, Recall: 0.6852, F1-measure: 0.6746


For threshold:  0.4
Micro-average quality numbers
Precision: 0.7360, Recall: 0.6227, F1-measure: 0.6747


For threshold:  0.5
Micro-average quality numbers
Precision: 0.7864, Recall: 0.5669, F1-measure: 0.6588


For threshold:  0.6
Micro-average quality numbers
Precision: 0.8280, Recall: 0.5135, F1-measure: 0.6339


For threshold:  0.7
Micro-average quality numbers
Precision: 0.8672, Recall: 0.4638, F1-measure: 0.6044


For threshold:  0.8
Micro-average quality numbers
Precision: 0.8977, Recall: 0.3977, F1-measure: 0.5512


For threshold:  0.9
Micro-average quality numbers
Precision: 0.9277, Recall: 0.3087, F1-measure: 0.4633




## For 300d Model

In [41]:
df_valid = pd.concat([X_valid_Q1, y_valid_Q1.iloc[:,:12]], axis = 1)

# pre-processing test data
from tqdm import tqdm
preprocessed_synopsis = []
# tqdm is for printing the status bar
for sentance in df_valid['Comment'].values:
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_synopsis.append(sentance.strip())
    
    
df_valid['preprocessed_comments']=preprocessed_synopsis

# creating X and Y
X_valid = df_valid[['preprocessed_comments']]
y_valid = df_valid.drop(columns=['Comment', 'preprocessed_comments'])

y_valid = np.array(y_valid)
# creating padded dataset for x_valid
encoded_docs_valid = vect.texts_to_sequences(X_valid['preprocessed_comments'])
padded_docs_valid = pad_sequences(encoded_docs_valid, maxlen=max_len, padding='post')


score = model.evaluate(padded_docs_valid,y_valid)
score

2594/2594 [==============================] - 591s 228ms/step


[0.20097736289341264, 0.9246659160303721]

In [42]:
pred = model.predict(padded_docs_valid, batch_size=batch_size, verbose=1)

2594/2594 [==============================] - 828s 319ms/step


In [43]:
from sklearn.metrics import precision_score, recall_score, f1_score

#predictions=model.predict([padded_docs_test])
predictions = pred
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_valid, pred, average='micro')
    recall = recall_score(y_valid, pred, average='micro')
    f1 = f1_score(y_valid, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))
    print('\n')

For threshold:  0.1
Micro-average quality numbers
Precision: 0.4024, Recall: 0.8256, F1-measure: 0.5411


For threshold:  0.2
Micro-average quality numbers
Precision: 0.5757, Recall: 0.6960, F1-measure: 0.6301


For threshold:  0.3
Micro-average quality numbers
Precision: 0.6917, Recall: 0.5959, F1-measure: 0.6402


For threshold:  0.4
Micro-average quality numbers
Precision: 0.7718, Recall: 0.5133, F1-measure: 0.6165


For threshold:  0.5
Micro-average quality numbers
Precision: 0.8269, Recall: 0.4450, F1-measure: 0.5786


For threshold:  0.6
Micro-average quality numbers
Precision: 0.8701, Recall: 0.3889, F1-measure: 0.5375


For threshold:  0.7
Micro-average quality numbers
Precision: 0.8982, Recall: 0.3317, F1-measure: 0.4845


For threshold:  0.8
Micro-average quality numbers
Precision: 0.9351, Recall: 0.2670, F1-measure: 0.4154


For threshold:  0.9
Micro-average quality numbers
Precision: 0.9658, Recall: 0.1954, F1-measure: 0.3251


